In [3]:
file_path = r'C:\Users\sneha\Downloads\cps_00006.csv.gz'

In [4]:
import pandas as pd
import numpy as np
import gzip

# data preperation code


def cps_data(file_path):
  
    df = pd.read_csv(file_path, compression='gzip', header=0) # importing the data

    
    df = df[(df['INCWAGE'] != 99999999) & (df['INCWAGE'] != 0) & (df['INCWAGE'] != 999)]  # dropping the rows containing invalid and 0 INCWAGE according to the labels

    df['INCWAGE'] = np.log(df['INCWAGE'])  # taking log of weekly earnings which will be our dependent variable in the regressions


    df = df[(df['EDUC'] != 0) & (df['EDUC'] != 1)]  # dropping education levels 0 and 1 as they are invalid entries as per the labels of the dataset

    df = df[(df['YEAR'] >= 1980) & (df['YEAR'] <= 2000)]  # taking the time frame from 1980 to 2000

    def categorize_education(educ_code): # creating a fucntion to categorize education levels
        if educ_code <= 10:
            return 'Up to Grade 10'
        elif 10 < educ_code <= 70:
            return 'High School'
        elif 70 < educ_code <= 123:
            return "Master's Degree"
        else:
            return 'Doctorate Degree'

    
    df['Education_Category'] = df['EDUC'].apply(categorize_education) # applying the function to create a new 'Education_Category' column

    df = pd.get_dummies(df, columns=['Education_Category'], prefix='', prefix_sep='', drop_first=True)

    boolean = ['Up to Grade 10', 'High School', "Master's Degree"]
    df[boolean] = df[boolean].astype(int)

    df = df[~((df['STATEFIP'] > 56) | (df['STATEFIP'] == 11))]  # taking only the 50 states of the States and exclusing the regions as per the labels of the dataset

    df = df[(df['AGE'] >= 25) & (df['AGE'] <= 50)]  # taking the age group from 25 to 50

    df = df[df['SEX'] == 2] # taking only female respondents

    return df

In [5]:

import statsmodels.api as sm

# data aggregation code

def process_cps_data(file_path):

    df = cps_data(file_path)

    X = df[['High School', "Master's Degree", 'Up to Grade 10', 'AGE']] # the covariates used 1st stage of data aggregation
    y = df['INCWAGE']

    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    y_pred = model.predict(X)  # obtaining the predicted values of the model

    residuals = y - y_pred

    df['Residuals'] = residuals

    residuals_mean_by_state_year = df.groupby(['STATEFIP', 'YEAR'])['Residuals'].mean().reset_index()

    residuals_mean_by_state_year 

    return residuals_mean_by_state_year

In [13]:
# wage correlogram regression


data = process_cps_data(file_path)  # importing the processed and aggregated data

 
dummy_df = pd.get_dummies(data['STATEFIP'], prefix='STATEFIP', drop_first=True).astype(int)

    
data = pd.concat([data, dummy_df], axis=1)   # Concatenate the dummy variables with the original DataFrame

dummy_df2 = pd.get_dummies(data['YEAR'], prefix='YEAR', drop_first=True).astype(int)

    
data = pd.concat([data, dummy_df2], axis=1)   # Concatenate the dummy variables with the original DataFrame

X = data[['STATEFIP_2', 'STATEFIP_4',
       'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9', 'STATEFIP_10',
       'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15', 'STATEFIP_16',
       'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19', 'STATEFIP_20',
       'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23', 'STATEFIP_24',
       'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27', 'STATEFIP_28',
       'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31', 'STATEFIP_32',
       'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35', 'STATEFIP_36',
       'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39', 'STATEFIP_40',
       'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44', 'STATEFIP_45',
       'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48', 'STATEFIP_49',
       'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53', 'STATEFIP_54',
       'STATEFIP_55', 'STATEFIP_56', 'YEAR_1981', 'YEAR_1982', 'YEAR_1983',
       'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987', 'YEAR_1988',
       'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992', 'YEAR_1993',
       'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997', 'YEAR_1998',
       'YEAR_1999', 'YEAR_2000']]

X = sm.add_constant(X)
Y = data['Residuals'] 
model = sm.OLS(Y, X).fit()

data['Resid'] = model.resid

df = data.copy()

df['Resid_lag1'] = df.groupby('STATEFIP')['Resid'].shift(1)  # getting the lagged value of the residuals
df['Resid_lag2'] = df.groupby('STATEFIP')['Resid'].shift(2)
df['Resid_lag3'] = df.groupby('STATEFIP')['Resid'].shift(3)


df.dropna(inplace=True)   # Drop rows with NaN values (due to shifting)

X = df[['Resid_lag1', 'Resid_lag2', 'Resid_lag3']]    
y = df['Resid']  

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())






                            OLS Regression Results                            
Dep. Variable:                  Resid   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     51.02
Date:                Thu, 22 Feb 2024   Prob (F-statistic):           1.90e-30
Time:                        22:21:27   Log-Likelihood:                 1145.6
No. Observations:                 900   AIC:                            -2283.
Df Residuals:                     896   BIC:                            -2264.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.157e-15      0.002  -9.53e-13      1.0